# Analysis of Bioacoustic Data

This notebook provides tools for analyzing data using a custom classifier (developed with `agile_modeling.ipynb`).

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import collections
from etils import epath
from ml_collections import config_dict
import pandas as pd
from chirp.inference import tf_examples
from chirp.inference.search import bootstrap
from chirp.inference.classify import classify
from perch_hoplite.zoo import zoo_interface
from pydub import AudioSegment
import pandas as pd

In [2]:
# Specify species your working on
target_class = tuple(pd.read_csv("/workspaces/2023_ECCC4_Biodiv/data/custom_classifier/class_list.csv")['custom'].tolist())
#target_class = "Animals"

# Keep the negative logits?
keep_non_detections = False

# Set True if using class specific thresholding
class_specific_threshold = True

# Working dir for saving products
working_dir = "/workspaces/2023_ECCC4_Biodiv/data/"
embeddings_path = working_dir + "embeddings"
custom_classifier_path = epath.Path(working_dir) / "custom_classifier"

# Path to save predictions csv
output_filepath = working_dir + "inference_top_44_species/"
os.makedirs(output_filepath, exist_ok=True)

## Load model state

In [3]:
# Load Existing Project State and Models.
if (embeddings_path or (epath.Path(working_dir) / 'embeddings/config.json').exists()):
  if not embeddings_path:
    # Use the default embeddings path, as it seems we found a config there.
    embeddings_path = epath.Path(working_dir) / 'embeddings'
  # Get relevant info from the embedding configuration.
  bootstrap_config = bootstrap.BootstrapConfig.load_from_embedding_path(
      embeddings_path=embeddings_path,
      annotated_path=epath.Path(""))
else:
  raise ValueError('No embedding configuration found.')

project_state = bootstrap.BootstrapState(
    bootstrap_config, baw_auth_token='')

cfg = config_dict.ConfigDict({
    'model_path': custom_classifier_path,
    'logits_key': 'custom',
})
logits_head = zoo_interface.LogitsOutputHead.from_config(cfg)
model = logits_head.logits_model
class_list = logits_head.class_list
print('Loaded custom model with classes: ')
print('\t' + '\n\t'.join(class_list.classes))

Loaded custom model with classes: 
	Anas platyrhynchos
	Branta canadensis
	Cardinalis cardinalis
	Catharus fuscescens
	Certhia americana
	Colaptes auratus
	Contopus virens
	Corthylio calendula
	Corvus corax
	Cyanocitta cristata
	Dryobates pubescens
	Dryobates villosus
	Dryocopus pileatus
	Dumetella carolinensis
	Empidonax alnorum
	Empidonax minimus
	Geothlypis trichas
	Hylocichla mustelina
	Icterus galbula
	Junco hyemalis
	Larus delawarensis
	Melospiza georgiana
	Melospiza melodia
	Mniotilta varia
	Pheucticus ludovicianus
	Piranga olivacea
	Poecile atricapillus
	Regulus satrapa
	Sayornis phoebe
	Seiurus aurocapilla
	Setophaga fusca
	Setophaga magnolia
	Setophaga pensylvanica
	Setophaga petechia
	Setophaga ruticilla
	Setophaga virens
	Sitta carolinensis
	Spinus tristis
	Tachycineta bicolor
	Troglodytes hiemalis
	Turdus migratorius
	Vireo gilvus
	Vireo philadelphicus
	Zonotrichia albicollis


## Use trained model to make predictions

In [4]:
# This cell writes detections (locations of audio windows where
# the logit was greater than a threshold) to a CSV file.
#output_filepath_name = output_filepath + target_class + ".csv"
output_filepath_name = output_filepath  + "custom_logit_threshold.csv"

# Set the default detection thresholds, used for all classes.
# To set per-class detection thresholds, modify the code below.
# Keep in mind that thresholds are on the logit scale, so 0.0
# corresponds to a 50% model confidence.
default_threshold = 0.0

if default_threshold is None:
  # In this case, all logits are written. This can lead to very large CSV files.
  class_thresholds = None
else:
  class_thresholds = collections.defaultdict(lambda: default_threshold)
  for tar in target_class:
    class_thresholds[tar] = default_threshold

if class_specific_threshold:
  # Set per-class thresholds here.
  class_thresholds = collections.defaultdict(lambda: default_threshold)
  class_thresholds['Anas platyrhynchos'] = 0.0
  class_thresholds['Branta canadensis'] = 0.0
  class_thresholds['Cardinalis cardinalis'] = 0.0
  class_thresholds['Catharus fuscescens'] = 0.0
  class_thresholds['Certhia americana'] = 0.7
  class_thresholds['Charadrius vociferus'] = 0.0
  class_thresholds['Colaptes auratus'] = 0.0
  class_thresholds['Contopus virens'] = 0.0
  class_thresholds['Corthylio calendula'] = 0.0
  class_thresholds['Corvus brachyrhynchos'] = 0.0
  class_thresholds['Corvus corax'] = 0.0
  class_thresholds['Cyanocitta cristata'] = 0.0
  class_thresholds['Dolichonyx oryzivorus'] = 0.0
  class_thresholds['Dryobates pubescens'] = 0.0
  class_thresholds['Dryobates villosus'] = 0.0
  class_thresholds['Dryocopus pileatus'] = 0.0
  class_thresholds['Dumetella carolinensis'] = 0.0
  class_thresholds['Empidonax alnorum'] = 0.0
  class_thresholds['Empidonax flaviventris'] = 0.0
  class_thresholds['Empidonax minimus'] = 0.0
  class_thresholds['Geothlypis trichas'] = 0.0
  class_thresholds['Hirundo rustica'] = 0.0
  class_thresholds['Hylocichla mustelina'] = 0.0
  class_thresholds['Icterus galbula'] = 0.4
  class_thresholds['Junco hyemalis'] = 0.0
  class_thresholds['Larus delawarensis'] = 0.0
  class_thresholds['Leiothlypis ruficapilla'] = 0.0
  class_thresholds['Melospiza georgiana'] = 0.0
  class_thresholds['Melospiza melodia'] = 0.0
  class_thresholds['Mniotilta varia'] = 0.0
  class_thresholds['Pheucticus ludovicianus'] = 0.0
  class_thresholds['Piranga olivacea'] = 0.0
  class_thresholds['Podilymbus podiceps'] = 0.0
  class_thresholds['Poecile atricapillus'] = 0.0
  class_thresholds['Regulus satrapa'] = 0.0
  class_thresholds['Sayornis phoebe'] = 0.0
  class_thresholds['Seiurus aurocapilla'] = 0.0
  class_thresholds['Setophaga caerulescens'] = 0.0
  class_thresholds['Setophaga fusca'] = 0.0
  class_thresholds['Setophaga magnolia'] = 0.0
  class_thresholds['Setophaga pensylvanica'] = 0.0
  class_thresholds['Setophaga petechia'] = 0.0
  class_thresholds['Setophaga ruticilla'] = 0.0
  class_thresholds['Setophaga striata'] = 0.0
  class_thresholds['Setophaga tigrina'] = 0.0
  class_thresholds['Setophaga virens'] = 0.95
  class_thresholds['Sitta carolinensis'] = 0.0
  class_thresholds['Spinus pinus'] = 0.0
  class_thresholds['Spinus tristis'] = 0.0
  class_thresholds['Sturnella magna'] = 0.0 
  class_thresholds['Tachycineta bicolor'] = 0.0
  class_thresholds['Troglodytes hiemalis'] = 0.0
  class_thresholds['Turdus migratorius'] = 0.0
  class_thresholds['Vireo gilvus'] = 0.0
  class_thresholds['Vireo philadelphicus'] = 0.0
  class_thresholds['Vireo solitarius'] = 0.0
  class_thresholds['Zonotrichia albicollis'] = 0.0

# Classes to ignore when counting detections.
exclude_classes = [] 

# The `include_classes` list is ignored if empty.
# If non-empty, only scores for these classes will be written.
include_classes = []

In [5]:
# Make predictions
embeddings_ds = tf_examples.create_embeddings_dataset(
    embeddings_path, file_glob='embeddings-*')

classify.write_inference_csv(
    embeddings_ds=embeddings_ds,
    model=logits_head,
    labels=class_list.classes,
    output_filepath=output_filepath_name,
    threshold=class_thresholds,
    embedding_hop_size_s=bootstrap_config.embedding_hop_size_s,
    include_classes=include_classes,
    exclude_classes=exclude_classes,
    keep_non_detections=keep_non_detections)

# large dataset (e.g. 1000 observations)  = 600 detections
# smaller random = 408
# smaller top = 184 
# larger top (44species) = 352

50it [00:00, 476.66it/s]




   Detection count:  154
NonDetection count:  26246


## Move correctly predicted labels

In [ ]:
# Load file containing predictions
predictions = pd.read_csv(output_filepath_name)

# Filter to target class
predictions = predictions[predictions.iloc[:,2] == target_class]

# Sort highest to lowest logits
predictions = predictions.sort_values(by=" logit", ascending=False)

# Select top 100 examples
predictions = predictions.head(100)

# Path to move files
label_location =  working_dir + "labels_noise/" + target_class + target_class

# Loop over rows, extract snippets and save to folder
for _, row in predictions.iterrows():
    file_path = row["filename"]  # Full path to the WAV file
    timestamp = row[" timestamp_s"]  # Start time in seconds

    # Load the WAV file
    audio = AudioSegment.from_wav(file_path)

    # Compute start and end times in milliseconds
    start_time = int(timestamp * 1000)
    end_time = start_time + 5000  # 5 seconds later

    # Clip the audio
    clipped_audio = audio[start_time:end_time]

    # Extract original file name without path
    base_name = os.path.basename(file_path)  # e.g., "20231113_234000.WAV"
    name, ext = os.path.splitext(base_name)  # Split into name and extension

    # Create new filename with timestamp appended
    new_filename = f"{target_class}_{name}_{int(timestamp)}.wav"
    output_path = os.path.join(label_location, new_filename)

    # Save the clipped audio
    clipped_audio.export(output_path, format="wav")